In [2]:
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as F

from pyspark.sql.types import (
    ArrayType,
    IntegerType,
    MapType,
    StringType,
    StructField,
    StructType,
    FloatType,
    TimestampType,
    BooleanType,
    DateType
)

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [3]:
def generate_schema():
    """
    JSON schema went through a few modifications across different versions of the dataset.
    In this work, we're finalizing the dataset version 111 and using its schema as final.
    The schema provided along with the dataset acts as a reference point however there were
    a few falsely nested structures in it which has been corrected by us. Further, there are
    quite a lot of fields but we'll extract only the ones we need for our project.
    
    Original schema:
    https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-13/json_schema.txt
    """
    
    # Extract author information, although not needed for modelling but good to have for data analysis
    authors_schema = ArrayType(
        StructType(
            [
                StructField("first", StringType()),
                StructField("middle", ArrayType(StringType())),
                StructField("last", StringType()),
                StructField("suffix", StringType())
            ]
        )
    )
    
    # Extract different sections such as abstract and body text
    section_schema = ArrayType(
        StructType(
            [
                StructField("text", StringType()),
                StructField("section", StringType())
            ]
        )
    )

    schema = StructType(
        [
            StructField("paper_id", StringType()),
            StructField(
                "metadata",
                StructType(
                    [
                        StructField("title", StringType()),
                        StructField("authors", authors_schema)
                    ]
                ),
                True,
            ),
            StructField("abstract", section_schema),
            StructField("body_text", section_schema),
        ]
    )
    
    return schema


def jsons_to_df(spark, path):
    return spark.read.json(path, schema=generate_schema(), multiLine=True)

In [86]:
spark = SparkSession.builder.appName('BDP').getOrCreate()
df = jsons_to_df(spark, path="/user/mnis/cord19/jsons")

# df.printSchema()

# Get title
df = df.withColumn('json_title', F.col('metadata.title'))

# Get authors
df = df.withColumn('firstnames', F.col('metadata.authors.first'))  # First name
df = df.withColumn('middlenames', F.col('metadata.authors.middle'))  # Middle name

# Middle names are list of list. Convert it to a list of strings
@F.udf(returnType=ArrayType(StringType()))
def parse_middlenames(array):
    return [" ".join(w).strip() for w in array]

df = df.withColumn('middlenames', parse_middlenames("middlenames"))
df = df.withColumn('lastnames', F.col('metadata.authors.last'))  # Last name
df = df.withColumn('suffixes', F.col('metadata.authors.suffix'))  # Suffix name

# Concat the first, middle, last and suffix names for each author
df = df.withColumn("json_authors", F.expr(
    "transform(firstnames, (x, i) -> concat(x, ' ', middlenames[i], ' ', lastnames[i], ' ', suffixes[i]))"))

# Concat the list of authors into one "; " separated string
df = df.withColumn('json_authors', F.concat_ws("; ", F.expr("transform(json_authors, x -> trim(x))")))

# Remove additional empty spaces from the names
@F.udf()
def parse_authors(array):
    return " ".join([x for x in array.split() if x.strip()])

df = df.withColumn('json_authors', F.concat(parse_authors("json_authors")))

# Get abstract
df = df.withColumn('json_abstract', F.col('abstract.text'))
df = df.withColumn('json_abstract', F.concat_ws(". ", 'json_abstract'))

# Get body text
df = df.withColumn('body_text', F.col('body_text.text'))
df = df.withColumn('body_text', F.concat_ws(". ", 'body_text'))

df = df.drop(*["metadata", "abstract", "firstnames", "middlenames", "lastnames", "suffixes"])

df.printSchema()

root
 |-- paper_id: string (nullable = true)
 |-- body_text: string (nullable = false)
 |-- json_title: string (nullable = true)
 |-- json_authors: string (nullable = true)
 |-- json_abstract: string (nullable = false)



In [88]:
df.select(["paper_id", "json_authors"]).show(20, truncate=80)

+----------------------------------------+--------------------------------------------------------------------------------+
|                                paper_id|                                                                    json_authors|
+----------------------------------------+--------------------------------------------------------------------------------+
|0036b28fddf7e93da0970303672934ea2f9944e7|B Ruggerone; A C Manchester; F Del Baldo; F Fracassi; J A Lidbury; J M Steine...|
|00548876f3e2dada41101d0de4dd2ceb6d1e38ac|                                                                                |
|00974a6604d8f6cd454f8dcf3436326c3e776af1|                                                                                |
|0061fcb1dd3a378cd8e5151925edc36146b2956f|Pedro F N Souza; Felipe P Mesquita; Jackson L Amaral; Patrícia G C Landim; Ka...|
|0085116cf3733157179e21a746782450c397e75c|Marco Contardi; Martina Lenzuni; Fabrizio Fiorentini; Maria Summa; Rosalia Be...|
|0034852

In [114]:
metadata = spark.read.csv("/user/mnis/cord19/csvs/metadata.csv", inferSchema=True, header=True)
metadata.count()

1056660

In [115]:
metadata = metadata.withColumnRenamed("authors", "metadata_authors")
metadata = metadata.withColumnRenamed("abstract", "metadata_abstract")
metadata = metadata.withColumnRenamed("title", "metadata_title")

In [91]:
metadata = metadata.dropna(how="all", subset=["pdf_json_files", "pmc_json_files"])
metadata.count()

409689

In [116]:
@F.udf(returnType='int')
def count_semicolons(s):
    return str(s).count(";")

metadata = metadata.withColumn("pdf_count", count_semicolons("pdf_json_files"))
metadata = metadata.withColumn("pmc_count", count_semicolons("pmc_json_files"))

metadata = metadata.filter((metadata.pdf_count == 0) | (metadata.pmc_count == 0))
metadata.count()

1056542

In [117]:
@F.udf()
def split_path(s):
    if s:
        return s.split("/")[-1].split(".json")[0].split(".xml")[0]
    return s

metadata = metadata.withColumn("pdf_json_files", split_path("pdf_json_files"))
metadata = metadata.withColumn("pmc_json_files", split_path("pmc_json_files"))

metadata = metadata.drop(*["sha", "license", "mag_id", "who_covidence_id", "url", "s2_id",
                           "pdf_count", "pmc_count"])

metadata.show(5)

+--------+--------+--------------------+--------------------+--------+---------+--------------------+------------+--------------------+--------------+--------+--------------------+--------------+
|cord_uid|source_x|      metadata_title|                 doi|   pmcid|pubmed_id|   metadata_abstract|publish_time|    metadata_authors|       journal|arxiv_id|      pdf_json_files|pmc_json_files|
+--------+--------+--------------------+--------------------+--------+---------+--------------------+------------+--------------------+--------------+--------+--------------------+--------------+
|ug7v899j|     PMC|Clinical features...|10.1186/1471-2334...|PMC35282| 11472636|OBJECTIVE: This r...|  2001-07-04|Madani, Tariq A; ...|BMC Infect Dis|    null|d1aafb70c066a2068...|      PMC35282|
|02tnwd4m|     PMC|Nitric oxide: a p...|        10.1186/rr14|PMC59543| 11667967|Inflammatory dise...|  2000-08-15|Vliet, Albert van...|    Respir Res|    null|6b0567729c2143a66...|      PMC59543|
|ejv2xln0|     PMC|S

In [94]:
metadata.printSchema()

root
 |-- cord_uid: string (nullable = true)
 |-- source_x: string (nullable = true)
 |-- metadata_title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubmed_id: string (nullable = true)
 |-- metadata_abstract: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- metadata_authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- arxiv_id: string (nullable = true)
 |-- pdf_json_files: string (nullable = true)
 |-- pmc_json_files: string (nullable = true)



In [95]:
metadata.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in metadata.columns]).show()

+--------+--------+--------------+-----+-----+---------+-----------------+------------+----------------+-------+--------+--------------+--------------+
|cord_uid|source_x|metadata_title|  doi|pmcid|pubmed_id|metadata_abstract|publish_time|metadata_authors|journal|arxiv_id|pdf_json_files|pmc_json_files|
+--------+--------+--------------+-----+-----+---------+-----------------+------------+----------------+-------+--------+--------------+--------------+
|       0|       0|             6|24636|55179|    91757|            55603|         102|            7098|  30294|  377023|         17810|         76312|
+--------+--------+--------------+-----+-----+---------+-----------------+------------+----------------+-------+--------+--------------+--------------+



In [118]:
metadata = metadata.withColumn("paper_id", F.when(F.col("pmc_json_files").isNull() | F.isnan("pmc_json_files"),
                                                 metadata.pdf_json_files).otherwise(metadata.pmc_json_files))

In [119]:
merged = metadata.join(df, on="paper_id", how="inner")
merged = merged.withColumn("title",
                            F.when(F.isnan("metadata_title") | F.col("metadata_title").isNull(),
                                   merged.json_title).otherwise(merged.metadata_title))
merged = merged.withColumn("abstract",
                            F.when(F.isnan("metadata_abstract") | F.col("metadata_abstract").isNull(),
                                   merged.json_abstract).otherwise(merged.metadata_abstract))
merged = merged.withColumn("authors",
                            F.when(F.isnan("metadata_authors") | F.col("metadata_authors").isNull(),
                                   merged.json_authors).otherwise(merged.metadata_authors))

merged = merged.drop(*["metadata_title", "json_title", "metadata_abstract",
                       "json_abstract", "metadata_authors", "json_authors"])

In [120]:
new = merged.toPandas()

In [121]:
new.shape

(176, 15)

In [112]:
new

,paper_id,cord_uid,source_x,doi,pmcid,pubmed_id,publish_time,journal,arxiv_id,pdf_json_files,pmc_json_files,body_text,title,abstract,authors
0,002ac316a3c1c3418eca3a4f1e144f7ca139767d,7bkib0m9,Medline; PMC,10.1177/21501319221097672,PMC9150224,35619243,2022-05-26,J Prim Care Community Health,None,002ac316a3c1c3418eca3a4f1e144f7ca139767d,None,Churches and faith-based organizations are a t...,The Black Church and Public Health: A Key Part...,The COVID-19 epidemic has negatively impacted ...,"Brown, Natasha R.; Alick, Candice L.; Heaston,..."
1,0015915e6bf0bdfd8c5b72cd584da735e0346c14,y0c39p7a,Medline; PMC,10.1007/978-0-387-71724-1_4,PMC7120101,18020305,2008,Reviews of Environmental Contamination and Tox...,None,0015915e6bf0bdfd8c5b72cd584da735e0346c14,None,The quality of drinking water in the United St...,Risk of Waterborne Illness Via Drinking Water ...,The quality of drinking water in the United St...,"Reynolds, Kelly A.; Mena, Kristina D.; Gerba, ..."
2,00363c053d290ced5cec482aaaf84f39ae037481,mj16c559,BioRxiv; WHO,10.1101/2020.08.19.225854,None,None,2020-08-21,bioRxiv,None,00363c053d290ced5cec482aaaf84f39ae037481,None,SARS-CoV-2 is a single stranded positive sense...,Iota-carrageenan and Xylitol inhibit SARS-CoV-...,COVID-19 (coronavirus disease 2019) is a pande...,"Bansal, Shruti; Jonsson, Colleen B.; Taylor, S..."
3,008dc26e3654788b9ed56a0026da4ab2d41bbf14,csaxxx2h,Medline; PMC,10.1111/famp.12577,PMC7361313,32621312,2020-07-03,Fam Process,None,008dc26e3654788b9ed56a0026da4ab2d41bbf14,None,The musings of a family therapist in Asia when...,The musings of a family therapist in Asia when...,"In response to the COVID‐19 crisis in Asia, a ...","Lee, Wai‐Yung"
4,00467bd1940aae7539467e3ae56a8210fd44fc80,okdilram,PMC,10.1007/3-7908-1624-8_68,PMC7121047,None,2005,Wirtschaftsinformatik 2005,None,00467bd1940aae7539467e3ae56a8210fd44fc80,None,An important characteristic of today's busines...,Ontology Evolution: MEDLINE Case Study,With the rising importance of knowledge interc...,"Abcckcr, Andrcas; Stojanovic, Ljiljana"
5,000b0174f992cb326a891f756d4ae5531f2845f7,rm5g8t63,BioRxiv; MedRxiv,10.1101/574103,None,None,2019-03-11,bioRxiv,None,000b0174f992cb326a891f756d4ae5531f2845f7,None,Since the first human case of Middle East Resp...,A systematic review of MERS-CoV (Middle East R...,Human infection with Middle East Respiratory S...,"Dighe, Amy; Jombart, Thibaut; Van Kerkhove, Ma..."
6,003f0478223711437a50964fae472e3d5cb2659f,ycbag8oc,MedRxiv; WHO,10.1101/2020.09.08.20190603,None,None,2020-09-10,None,None,003f0478223711437a50964fae472e3d5cb2659f,None,Considering the stressful routine and risk of ...,"Sleep Disturbances, Anxiety, and Burnout durin...",Study objectives: To evaluate the impact of CO...,"Drager, L.; Pachito, D.; Moreno, C.; Tavares, ..."
7,00617cc710d93dc76591af8065d5f117536dacec,nizs968z,Elsevier; Medline; PMC; WHO,10.1016/j.ijid.2022.02.057,PMC8894685,35248715,2022-03-04,Int J Infect Dis,None,00617cc710d93dc76591af8065d5f117536dacec,None,The World Health Organization (WHO) and STOP T...,World TB Day 2022: Revamping and Reshaping Glo...,,"Petersen, Eskild; Al-Abri, Seif; Chakaya, Jere..."
8,006df059f8fb6b80c15094c6ef623c4df8565784,npawv8i4,ArXiv,10.1145/3430665.3456370,None,None,2021-04-23,None,2104.11806,006df059f8fb6b80c15094c6ef623c4df8565784,None,Open-ended projects can prove very difficult f...,Novices' Learning Barriers When Using Code Exa...,Open-ended programming increases students' mot...,"Wang, Wengran; Kwatra, Archit; Skripchuk, Jame..."
9,00339c93e11141d71c66e8562f5cb4020e6def2c,ija2pb97,PMC,10.1007/s00390-003-0356-5,PMC7101948,32287630,2003,Intensivmed Notfallmed,None,00339c93e11141d71c66e8562f5cb4020e6def2c,None,n Summary The number of HIVinfected patients i...,HIV- und AIDS-Patienten auf der Intensivstation,The number of HIV-infected patients in Germany...,"Mandraka, Falitsa; Salzberger, Bernd; Glück, T..."
